https://clay-atlas.com/blog/2020/06/30/pytorch-%E5%A6%82%E4%BD%95%E4%BD%BF%E7%94%A8-hugging-face-%E6%89%80%E6%8F%90%E4%BE%9B%E7%9A%84-transformers-%E4%BB%A5-bert-%E7%82%BA%E4%BE%8B/

In [1]:
!pip3 freeze | grep torch

torch==1.7.1
torchsummary==1.5.1
torchtext==0.8.0
torchvision==0.7.0


In [2]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len_token = tokenizer.max_model_input_sizes['bert-base-uncased']
max_len_token = 128
# print(max_len_token)

In [3]:
# def tokenize_cut(text):
#     tokens = tokenizer.tokenize(text)
#     embedded = tokenizer.convert_tokens_to_ids(tokens)
#     return embedded[:max_len_token]

def tokenize_cut(text):
    encoding = tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=max_len_token,
      return_token_type_ids=False,
      pad_to_max_length=True,
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
    )
    
    # return enodded idxs and self-attention masks
    return encoding['input_ids'].flatten(), encoding['attention_mask'].flatten()

In [4]:
# get all special token_id from bert tokenizer
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id
# print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

NameError: name 'model' is not defined

In [5]:
import os
import csv
import torch
from torch.utils.data import Dataset, DataLoader
class TwitterDataset(Dataset):
    def __init__(self, f_name='./twitter_sentiment', mode='train'):
        self.mode = mode
        
        text_list = []
        label_list = []
        
        f_path = os.path.join(f_name, '{}.csv'.format(mode))
        with open(f_path) as f:
            reader = csv.DictReader(f)
            for line in reader:
                text = line['text']
                label = int(line['sentiment_label'])
                text_list.append(text)
                label_list.append(label)

        self.text_list = text_list
        self.label_list = label_list
    
    def __getitem__(self, idx):
        text = self.text_list[idx]
        text, mask = tokenize_cut(text)
#         print('text', text)
#         print('mask', mask)
        
        if self.mode == 'test':
            return text, mask
        label = torch.tensor(self.label_list[idx])
        return text, mask, label

    def __len__(self):
        return len(self.text_list)

In [6]:
dataset_train = TwitterDataset(mode='train')

batch_size = 64
device = torch.device('cuda:2')

train_data = DataLoader(dataset_train, batch_size=batch_size)

In [7]:
# import torch
# from torchtext import data

# TEXT = data.Field(batch_first = True,
#                   use_vocab = False,
#                   tokenize = tokenize_cut,
#                   preprocessing = tokenizer.convert_tokens_to_ids,
#                   init_token = init_token_idx,
#                   eos_token = eos_token_idx,
#                   pad_token = pad_token_idx,
#                   unk_token = unk_token_idx)

# LABEL = data.LabelField()

# train_rawdata, test_rawdata = data.TabularDataset.splits(
#     path='twitter_sentiment', format='csv', skip_header=True,
#     train='train.csv', validation='val.csv',
#     fields=[
#         ('text', TEXT),
#         ('label', LABEL),
#     ]
# )

# LABEL.build_vocab(train_rawdata)

In [8]:
# batch_size = 64
# device = torch.device('cuda:2')

# train_data, test_data = data.BucketIterator.splits(
#     (train_rawdata, test_rawdata),
#     batch_size=batch_size,
#     sort_key=lambda x: len(x.text),
#     sort_within_batch=True,
#     device=device
# )

In [9]:
# from torch import nn
# import torch.nn.functional as F
# from transformers import BertModel

# class FineTuneBertModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.embedding = BertModel.from_pretrained('bert-base-uncased')
#         self.rnn =  nn.GRU(
#             input_size=768,
#             hidden_size=768,
#             dropout=0.3,
#             num_layers=5,
#             bidirectional=False,
#             batch_first=True,
#         )
        
#         self.fc = nn.Linear(768, 3)
    
#     def forward(self, x, masks=None):
# #         with torch.no_grad():
#         embedded = self.embedding(x, attention_mask=masks)
#         embedded = embedded[0]
# #         print('embbed', embbed)

#         _, embedded = self.rnn(embedded)
        
#         hidden = F.dropout(embedded[-1,:,:])
        
#         out = self.fc(hidden)
#         return out

In [10]:
from torch import nn
import torch.nn.functional as F
from transformers import BertModel

class FineTuneBertModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(inplace=True)
        self.fc = nn.Linear(768, 3)
    
    def forward(self, x, masks=None):
#         print('mask', masks)
#         with torch.no_grad():
        embedded = self.embedding(x, attention_mask=masks)
        embedded = embedded[0]
        
#         print('emb', embedded.shape)
        
        hidden = self.dropout(embedded[:,-1,:])
        
        out = self.fc(hidden)
        return out

In [11]:
model = FineTuneBertModel()

In [12]:
# for name, param in model.named_parameters():
# #     print(name)
#     if name.startswith('embedding'):
#         param.requires_grad = False

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,484,547 trainable parameters


In [14]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

In [15]:
model = model.to(device)
criterion = criterion.to(device)

In [16]:
# test data and model
text, mask, label = next(iter(train_data))
print('raw text', text)
print('raw label', label)
text = text.to(device)
mask = mask.to(device)
label = label.to(device)

out = model(text, mask)
print('out', out.shape)
# print('label', label.shape)
loss = criterion(out, label)
print(loss)

raw text tensor([[ 101, 1030, 4943,  ...,    0,    0,    0],
        [ 101, 1030, 6892,  ...,    0,    0,    0],
        [ 101, 1030, 2142,  ...,    0,    0,    0],
        ...,
        [ 101, 1030, 6892,  ...,    0,    0,    0],
        [ 101, 1030, 6892,  ...,    0,    0,    0],
        [ 101, 1030, 2142,  ...,    0,    0,    0]])
raw label tensor([0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 1, 0,
        0, 2, 0, 2, 1, 2, 0, 1, 2, 0, 2, 1, 0, 1, 1, 1])
out torch.Size([64, 3])
tensor(1.2077, device='cuda:2', grad_fn=<NllLossBackward>)


In [17]:
def accuracy(raw_preds, y):
    preds = raw_preds.argmax(dim=1)
    acc = (preds == y).float().mean().cpu()
    return acc

In [18]:
# # test acc function
# accuracy(out, d.label)

In [19]:
from tqdm import tqdm

def train(model, data, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for text, mask, label in tqdm(data, total=len(data)):
        text = text.to(device)
        mask = mask.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        pred = model(text, mask).squeeze(1)
        print(pred.shape)
        loss = criterion(pred, label)
        acc = accuracy(pred, label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(data), epoch_acc / len(data)

def test(model, data, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    for text, mask, label in tqdm(data, total=len(data)):
        pred = model(text).squeeze(1)
        loss = criterion(pred, label)
        acc = accuracy(pred, label)
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(data), epoch_acc / len(data)

In [20]:
max_epoches = 3
log_interval = 1
best_val_loss = float('inf')

for epoch in range(1, max_epoches + 1):
    train_loss, train_acc = train(model, train_data, optimizer, criterion)
#     test_loss, test_acc = test(model, test_data, criterion)
    
    if epoch % log_interval == 0:
        print('Epoch {} train_loss: {} train_acc: {}'.format(
            epoch, train_loss, train_acc
        ))

  0%|          | 0/161 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 2; 10.76 GiB total capacity; 9.67 GiB already allocated; 31.56 MiB free; 9.73 GiB reserved in total by PyTorch)

In [ ]:
from collections import defaultdict

counts = defaultdict(int)
total = 0

for batch in train_data:
    label = batch.label.tolist()
    for i in label:
        counts[i] += 1
    total += len(label)

print('counts', counts)
print('total', total)

In [ ]:
import numpy as np

counts = np.zeros(550)

f_name = './twitter_sentiment'
mode = 'train'
f_path = os.path.join(f_name, '{}.csv'.format(mode))
with open(f_path) as f:
    reader = csv.DictReader(f)
    for line in reader:
        text = line['text']
        l = len(text)
        counts[l] += 1

# print('counts', counts)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(200), counts[:200])
plt.show()